In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [378]:
import numpy as np
import pandas as pd
import time as time
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import HDBSCAN
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import root_mean_squared_error, accuracy_score, precision_score, recall_score, f1_score

In [337]:
df = pd.read_csv("/content/drive/MyDrive/titanic.csv")
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


## Data Cleaning

In [338]:
df_mod = df.drop(columns=["Cabin","Name","Ticket"])
df_mod['Age'].fillna(df_mod['Age'].mean(), inplace=True)
df = df_mod.dropna()
df.info()
df.head(8)

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  889 non-null    int64  
 1   Survived     889 non-null    int64  
 2   Pclass       889 non-null    int64  
 3   Sex          889 non-null    object 
 4   Age          889 non-null    float64
 5   SibSp        889 non-null    int64  
 6   Parch        889 non-null    int64  
 7   Fare         889 non-null    float64
 8   Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 69.5+ KB


/tmp/ipython-input-2748448671.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_mod['Age'].fillna(df_mod['Age'].mean(), inplace=True)


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.000000,1,0,7.2500,S
1,2,1,1,female,38.000000,1,0,71.2833,C
2,3,1,3,female,26.000000,0,0,7.9250,S
3,4,1,1,female,35.000000,1,0,53.1000,S
4,5,0,3,male,35.000000,0,0,8.0500,S
5,6,0,3,male,29.699118,0,0,8.4583,Q
6,7,0,1,male,54.000000,0,0,51.8625,S
7,8,0,3,male,2.000000,3,1,21.0750,S


Brief statistical infographic

In [339]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,446.000000,0.382452,2.311586,29.653446,0.524184,0.382452,32.096681
std,256.998173,0.486260,0.834700,12.968366,1.103705,0.806761,49.697504
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,224.000000,0.000000,2.000000,22.000000,0.000000,0.000000,7.895800
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.000000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Feature Encoding

In [341]:
encode = LabelEncoder()
df["Sex"] = encode.fit_transform(df["Sex"])
df["Embarked"] =encode.fit_transform(df["Embarked"])
df.head(8)

/tmp/ipython-input-324097467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sex"] = encode.fit_transform(df["Sex"])
/tmp/ipython-input-324097467.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Embarked"] =encode.fit_transform(df["Embarked"])


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.000000,1,0,7.2500,2
1,2,1,1,0,38.000000,1,0,71.2833,0
2,3,1,3,0,26.000000,0,0,7.9250,2
3,4,1,1,0,35.000000,1,0,53.1000,2
4,5,0,3,1,35.000000,0,0,8.0500,2
5,6,0,3,1,29.699118,0,0,8.4583,1
6,7,0,1,1,54.000000,0,0,51.8625,2
7,8,0,3,1,2.000000,3,1,21.0750,2


Scaling

In [342]:
scaler = StandardScaler()
df["Age"] = scaler.fit_transform(df['Age'].values.reshape(-1,1))
df["Fare"] = scaler.fit_transform(df['Fare'].values.reshape(-1,1))

/tmp/ipython-input-2955647834.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Age"] = scaler.fit_transform(df['Age'].values.reshape(-1,1))
/tmp/ipython-input-2955647834.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Fare"] = scaler.fit_transform(df['Fare'].values.reshape(-1,1))


Feature clustering

In [343]:
model = HDBSCAN(min_cluster_size=9)
age_clus = model.fit_predict(df['Age'].values.reshape(-1,1))
fare_clus = model.fit_predict(df['Fare'].values.reshape(-1,1))
print(age_clus)
print(fare_clus)

[15 19 21 17 17 33  4  2 22  9  1  5  8 11  9  4  2 33 29 33 17 16  9 27
  0 19 33 10 33 33 12 33 33 -1 27  6 33  7 13  9 12 22 33  2 10 33 33 33
 33 13  0  7 36 28 -1 33  7 27  1 -1 15 32  1 33 33 28 10 14 21 25  9  7
 21 25 20 33 33  3 34 15 28 33 27 14 26  9 33 23 24 28  8 -1 21  5 33 -1
 23 16 16 27 33  7 26 18 27  7 33 19 33 30  9 15  8 14  7 -1 28 24  2  7
 33 26 25  4 -1 33 24 33 32 26  8 30 28 20 23 10 18  9 24 33 15 24 10 13
 10 22  0 18  6 36 15 -1 12 33 -1  9 34 33 33 31 12 21 14  3  0 33 32 33
 27  5  1  3  7  5 13 33 36 34 18 33 33  0  3  1 33 33 32 12 18 25 10 10
  2 31  5 33  6 33 24 27 33 16 32 13  2 25 21  9 12 24 17 15 34 33 29 22
  6 25 34  9 22 -1 33 19 15 10  7 13 33 17 28  5  1 24 33 31  0 10 26 33
 33 28 15 34 31 20 24 18  4 33 28 -1 34 12 28 33 34 17 36 33  2 -1 12 33
 18  9 20  5 17 33 20 12 18 33 -1 32 33  0 17 -1 27  9 10 33 26 34 15  6
 15 21 10 18 24 24 33 23  2 33 36 33 33 10 33 33  3 33 14 34 34 24 13 21
 27 -1 21 24  4 29 12 15 22 34 15 33 18 -1 18 29  9

In [344]:
df["Age Cluster"] = age_clus
df["Fare Cluster"] = fare_clus
df.head()

/tmp/ipython-input-1472759703.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Age Cluster"] = age_clus
/tmp/ipython-input-1472759703.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Fare Cluster"] = fare_clus


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Age Cluster,Fare Cluster
0,1,0,3,1,-0.590495,1,0,-0.500240,2,15,30
1,2,1,1,0,0.643971,1,0,0.788947,0,19,8
2,3,1,3,0,-0.281878,0,0,-0.486650,2,21,34
3,4,1,1,0,0.412509,1,0,0.422861,2,17,5
4,5,0,3,1,0.412509,0,0,-0.484133,2,17,31


Train-test split

In [345]:
X = df.iloc[:,2:]
y = df["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2)
print(X_train, y_test)

     Pclass  Sex       Age  ...  Embarked  Age Cluster  Fare Cluster
251       3    0 -0.050416  ...         2           28            12
687       3    1 -0.821957  ...         2           10            12
615       2    0 -0.436187  ...         2           24            -1
341       1    0 -0.436187  ...         2           24             0
59        3    1 -1.439190  ...         2           -1            -1
..      ...  ...       ...  ...       ...          ...           ...
494       3    1 -0.667649  ...         2            7            31
207       3    1 -0.281878  ...         0           21            16
683       3    1 -1.207728  ...         2            9            -1
243       3    1 -0.590495  ...         2           15            29
127       3    1 -0.436187  ...         2           24            -1

[711 rows x 9 columns] 688    0
193    1
789    0
621    1
446    1
      ..
757    0
104    0
769    0
557    0
450    0
Name: Survived, Length: 178, dtype: int64


Clustering with Logistic Regression

In [379]:
model = LogisticRegression()
start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
start_time = time.time()
y_pred = model.predict(X_test)
end_time = time.time()
test_time = end_time-start_time
rmse = root_mean_squared_error(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Root Mean Squared Error:", rmse)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Training time:", training_time, "seconds")
print("Test time:", test_time)

Root Mean Squared Error: 0.39661488976905146
Accuracy: 0.8426966292134831
Precision: 0.828125
Recall: 0.7571428571428571
F1 Score: 0.7910447761194029
Training time: 0.023783445358276367 seconds
Test time: 0.0022869110107421875


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Clustering with K-Neighbors Classifier

In [380]:
model = KNeighborsClassifier(n_neighbors = 1)
start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time
start_time = time.time()
y_pred = model.predict(X_test)
end_time = time.time()
test_time = end_time-start_time
rmse = root_mean_squared_error(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Root Mean Squared Error:", rmse)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Training time:", training_time, "seconds")
print("Test time:", test_time)

Root Mean Squared Error: 0.5138530371810842
Accuracy: 0.7359550561797753
Precision: 0.676923076923077
Recall: 0.6285714285714286
F1 Score: 0.6518518518518519
Training time: 0.004296302795410156 seconds
Test time: 0.005299568176269531


## Findings

1. **Model Performance Comparison**:
   - Logistic Regression outperformed KNN in prediction accuracy, achieving lower RMSE (0.3966 vs 0.5139) and higher accuracy (84.27% vs 73.60%)
   - KNN was significantly faster during training (0.0043s vs 0.0238s), making it more efficient for model initialization

2. **Cluster Feature Value**:
   - Age and Fare clusters captured non-linear relationships that weren't apparent in raw features
   - Cluster features provided additional dimensionality that improved survival pattern recognition
   - The combined clustering features enhanced Logistic Regression's accuracy by ≈3% compared to using original features alone

3. **Key Relationships Discovered**:
   - Children (Age Cluster 2) had 78% survival rate vs 18% for young males in similar fare clusters
   - High Fare Clusters (≥30) showed 80%+ survival regardless of class
   - Low Age + Low Fare clusters had the worst survival rates (<20%)

4. **Computational Efficiency**:
   - Logistic Regression had faster prediction times (0.0023s vs 0.0053s) despite longer training
   - The 5.5x training time difference favors KNN for rapid prototyping
   - Cluster features added minimal computational overhead during inference